In [1]:
import perceval as pcvl
import numpy as np
import math

In [2]:
from perceval.components.unitary_components import PS, BS, PERM
from perceval.components.port import Port, Encoding
from perceval.utils import Encoding, PostSelect, Matrix
from perceval.components import Unitary

## The CZ gate in reference [5] - as a warm-up to the problem

In [ ]:
# Create 4 modes for the photons that has been encoded for the CZ operation, while the first two modes represent the |1> state of the first qbit, and the last two modes represent the herald qbits.
Knill_CZ = pcvl.Processor("SLOS", 4)

# Add the two pi-phase shifters to to the two system qubits
Knill_CZ.add(0, PS(phi=np.pi))
Knill_CZ.add(3, PS(phi=np.pi))

# Add the beam splitters
theta1 = 54.74 * 2 * np.pi / 180
theta2 = 17.63 * 2 * np.pi / 180

Knill_CZ.add((0, 2), BS.H(theta=theta1))
Knill_CZ.add((2, 3), BS.H(theta=theta1))
Knill_CZ.add((0, 2), BS.H(theta=-theta1))
Knill_CZ.add((2, 3), BS.H(theta=theta2))

# Add the heralds for the two beam splitters
Knill_CZ.add_herald(2,1)
Knill_CZ.add_herald(3,1)

# Since the the modes 0 and 1 are the |1> state of the control and target qubits respectively, we need to permute the modes to make the CZ gate act on the correct modes
CZ_gate = pcvl.Processor("SLOS", 4)
CZ_gate.add(1, PERM([1, 0]))
CZ_gate.add(2, Knill_CZ)
CZ_gate.add(1, PERM([1, 0]))

# Add ports
CZ_gate.add_port(0, Port(Encoding.DUAL_RAIL, 'ctrl'))
CZ_gate.add_port(2, Port(Encoding.DUAL_RAIL, 'data'))

pcvl.pdisplay(CZ_gate, recursive=True)

In [ ]:
# Analyze the performance of the newly-created CZ gate
processor = pcvl.Processor("SLOS", 4)
processor.add(2, pcvl.BS.H())
processor.add(0, CZ_gate) # This is the gate that we created
processor.add(2, pcvl.BS.H())

# Everything else is the same
states = {
    pcvl.BasicState([1, 0, 1, 0]): "00",
    pcvl.BasicState([1, 0, 0, 1]): "01",
    pcvl.BasicState([0, 1, 1, 0]): "10",
    pcvl.BasicState([0, 1, 0, 1]): "11"
}

ca = pcvl.algorithm.Analyzer(processor, states)

truth_table = {"00": "00", "01": "01", "10": "11", "11": "10"}
ca.compute(expected=truth_table)

pcvl.pdisplay(ca)
print(
    f"performance = {ca.performance}, fidelity = {ca.fidelity.real}")

## The CCZ gate in the Preceval library - serves as the bottom line

In [ ]:
processor = pcvl.Processor("SLOS", 6)
processor.add(4, pcvl.BS.H())
processor.add(0, pcvl.catalog["postprocessed ccz"].build_processor())
processor.add(4, pcvl.BS.H())

# Change the evaluation states to 3 qubits in dual-railed encoding
states = {
    pcvl.BasicState([1, 0, 1, 0, 1, 0]): "000",
    pcvl.BasicState([1, 0, 0, 1, 1, 0]): "010",
    pcvl.BasicState([0, 1, 1, 0, 1, 0]): "100",
    pcvl.BasicState([0, 1, 0, 1, 1, 0]): "110",
    pcvl.BasicState([1, 0, 1, 0, 0, 1]): "001",
    pcvl.BasicState([1, 0, 0, 1, 0, 1]): "011",
    pcvl.BasicState([0, 1, 1, 0, 0, 1]): "101",
    pcvl.BasicState([0, 1, 0, 1, 0, 1]): "111"
}

ca = pcvl.algorithm.Analyzer(processor, states)

# What is the truth table for the CCX (Tofolli) gate?
truth_table = {"000": "000", "010": "010", "100": "100", "110": "111", 
               "001": "001", "011": "011", "101": "101", "111": "110"}
ca.compute(expected=truth_table)

pcvl.pdisplay(ca)
print(
    f"performance = {ca.performance}, fidelity = {ca.fidelity.real}")

In [ ]:
# What does the pre-defined CCZ gate look like?
pcvl.pdisplay(pcvl.catalog["postprocessed ccz"].build_processor(), recursive=True)  

In [ ]:
# OK, I can't see anything inside. I'll copy their source code here. It is a unitary operator

m = Matrix([[0.509824528533959, 0, 0, 0, 0, 0, 0, 0, 0, 0.860278414296864, 0, 0],
                    [0, 0.509824528533959, 0, 0.321169327626332 + 0.556281593281541j, 0, 0, 0.330393705586394,
                        - 0.165196852793197 - 0.286129342288294j, -0.165196852793197 + 0.286129342288294j, 0, 0, 0],
                    [0, 0, 0.509824528533959, 0, 0, 0, 0, 0, 0, 0, 0.860278414296864, 0],
                    [0, 0, 0, 0.509824528533959, 0, 0.321169327626332 + 0.556281593281541j, -0.165196852793197
                        + 0.286129342288294j, 0.330393705586394, -0.165196852793197 - 0.286129342288294j, 0, 0, 0],
                    [0, 0, 0, 0, 0.509824528533959, 0, 0, 0, 0, 0, 0, 0.860278414296864],
                    [0, 0.321169327626332 + 0.556281593281541j, 0, 0, 0, 0.509824528533959, -0.165196852793197
                        - 0.286129342288294j, -0.165196852793197 + 0.286129342288294j, 0.330393705586394, 0, 0, 0],
                    [0, 0.330393705586394, 0, -0.165196852793197 - 0.286129342288294j, 0, -0.165196852793197
                        + 0.286129342288294j, -0.509824528533959, 0, -0.321169327626332 + 0.556281593281541j, 0, 0, 0],
                    [0, -0.165196852793197 + 0.286129342288294j, 0, 0.330393705586394, 0, -0.165196852793197
                        - 0.286129342288294j, -0.321169327626332 + 0.556281593281541j, -0.509824528533959, 0, 0, 0, 0],
                    [0, -0.165196852793197 - 0.286129342288294j, 0, -0.165196852793197 + 0.286129342288294j, 0,
                        0.330393705586394, 0, -0.321169327626332 + 0.556281593281541j, -0.509824528533959, 0, 0, 0],
                    [0.860278414296864, 0, 0, 0, 0, 0, 0, 0, 0, -0.509824528533959, 0, 0],
                    [0, 0, 0.860278414296864, 0, 0, 0, 0, 0, 0, 0, -0.509824528533959, 0],
                    [0, 0, 0, 0, 0.860278414296864, 0, 0, 0, 0, 0, 0, -0.509824528533959]])

# m is indeed a unitary matrix, up to the numerical precision
m_dagger = np.conjugate(m).T
m_dagger_m = np.dot(m_dagger, m)
m_dagger_m

# I guess the whole point for this challenge is to optimize m to give better performance, while keeping the fidelity same as 1.0

## Optimizing the CCZ gate - Getting serious now!

According to [6], `the maximum success probability is attained with minimal required resources!!`
In the above examples, there 6 herald modes used, which makes the whole matrix 12x12. According to [6], the minimum number of 3 heralds are needed to reach perfect fidelity. Let's optimize using the case where 3, 4, 5, 6 heralds, corresponding to 9x9, 10x10, 11x11, 12x12 matrices, respectively.

I will optimize the gate assuming that all heralds are |0>, because this minimizes the number of herald photons, the 3rd criteria for this challenge. **I think any case when a herald with |1> involved can be achieved by applying single-gates on the herald, which will not affect the overall success probability.** `Teammates, please think about this argument and see if it's a valid point!!`

In [3]:
# First, define the function to evaluate the matrix performance

def evaluate_matrix(m): # This function don't check M. make sure we input a squre matrix!
    dim = len(m) # figure out what is the dimension of the matrix
    q, r = np.linalg.qr(m) # QR decomposition. given a matrix m, we obtain q which we're sure is unitary. This serves the input of the function

    # The following part is essentially copied from the Quandela's source code
    processor = pcvl.Processor("SLOS", dim)
    processor.add(4, pcvl.BS.H()) # This is because mode 4 & 5 are the target qubit for the CCZ gate
    processor.add(0, Unitary(q))
    processor.add(4, pcvl.BS.H())

    # Post-selection. What does this mean? To be checked
    # It seems that this line is present on any 'post-selection' gates in the Quandela's source code, and is not present on 'herald' gates. Why? to be checked
    # The weird thing is that if I delete this line, the fidelity remains to be 1, and the performance almost doubles! Is this real? to be checked
    processor.set_postselection(PostSelect("[0,1]==1 & [2,3]==1 & [4,5]==1")) 

    processor.add_port(0, Port(Encoding.DUAL_RAIL, 'ctrl0'))
    processor.add_port(2, Port(Encoding.DUAL_RAIL, 'ctrl1'))
    processor.add_port(4, Port(Encoding.DUAL_RAIL, 'data'))

    for i in range(6, dim):
        processor.add_herald(i, 0)
    
    # Define the states and true tables for a CCNOT gate (after the Hadamard gates added on the data bit, CCZ gate has become a CCNOT gate and that's what we are going to evaluate)
    states = {
    pcvl.BasicState([1, 0, 1, 0, 1, 0]): "000",
    pcvl.BasicState([1, 0, 0, 1, 1, 0]): "010",
    pcvl.BasicState([0, 1, 1, 0, 1, 0]): "100",
    pcvl.BasicState([0, 1, 0, 1, 1, 0]): "110",
    pcvl.BasicState([1, 0, 1, 0, 0, 1]): "001",
    pcvl.BasicState([1, 0, 0, 1, 0, 1]): "011",
    pcvl.BasicState([0, 1, 1, 0, 0, 1]): "101",
    pcvl.BasicState([0, 1, 0, 1, 0, 1]): "111"
    }

    ca = pcvl.algorithm.Analyzer(processor, states)

    # What is the truth table for the CCNOT (Tofolli) gate?
    truth_table = {"000": "000", "010": "010", "100": "100", "110": "111", 
                   "001": "001", "011": "011", "101": "101", "111": "110"}
    ca.compute(expected=truth_table)
    # pcvl.pdisplay(ca)
    return ca.performance, ca.fidelity.real

In [4]:
# Let's optimize the function evaluate_matrix respect to m
# This is my first time ever to implement an advanced optimization algorithm to solve a real-world problem. I'm so excited! But please see if there is anything we an improve!
# I chose genetic algorithm. Not sure if this is the best choice

# Genetic Algorithm Components
class GeneticAlgorithm:
    def __init__(self, population_size, matrix_shape, mutation_rate, generations):
        self.population_size = population_size
        self.matrix_shape = matrix_shape
        self.mutation_rate = mutation_rate
        self.generations = generations
        self.population = [np.random.rand(*matrix_shape) for _ in range(population_size)]
    
    def fitness(self, matrix):
        a, b = evaluate_matrix(Matrix(matrix))
        return 10 * a - 1e7 * (1 - b)  # Maximizing a, giving a large peanalty for b < 1
    
    def select(self):
        # Simple tournament selection
        selected = []
        for _ in range(self.population_size):
            i, j = np.random.randint(0, self.population_size, 2)
            selected.append(self.population[i] if self.fitness(self.population[i]) > self.fitness(self.population[j]) else self.population[j])
        return selected
    
    def crossover(self, parent1, parent2):
        # Single point crossover
        if np.random.rand() < 0.5:  # Swap parents to ensure diversity
            parent1, parent2 = parent2, parent1
        split = np.random.randint(1, np.prod(self.matrix_shape))
        child1_flat = np.concatenate((parent1.flatten()[:split], parent2.flatten()[split:]))
        child2_flat = np.concatenate((parent2.flatten()[:split], parent1.flatten()[split:]))
        return child1_flat.reshape(self.matrix_shape), child2_flat.reshape(self.matrix_shape)
    
    def mutate(self, matrix):
        # Random mutation
        for i in range(matrix.shape[0]):
            for j in range(matrix.shape[1]):
                if np.random.rand() < self.mutation_rate:
                    matrix[i, j] += np.random.normal(0, 1)
        return matrix
    
    def evolve(self):
        for generation in range(self.generations):
            new_population = []
            selected = self.select()
            for i in range(0, self.population_size, 2):
                parent1, parent2 = selected[i], selected[i+1]
                child1, child2 = self.crossover(parent1, parent2)
                new_population.append(self.mutate(child1))
                new_population.append(self.mutate(child2))
            self.population = new_population
            # Optional: Print best fitness in generation
            best_fitness = max(self.fitness(matrix) for matrix in self.population)
            good_matrix = max(self.population, key=self.fitness)
            p, f = evaluate_matrix(Matrix(good_matrix))
            print(f"Generation {generation+1}: Best Fitness = {best_fitness}, Performance = {p}, Fidelity = {f}")
        return self.best_solution()
    
    def best_solution(self):
        best_matrix = max(self.population, key=self.fitness)
        return best_matrix, self.fitness(best_matrix)

In [5]:
# parameters of the genetic algorithm
population_size = 100
mutation_rate = 0.05
generations = 800

best_matrix = []
best_fitness = []

for dim in [9, 10, 11, 12]:
    print(dim)
    matrix_shape = (dim, dim)
    ga = GeneticAlgorithm(population_size, matrix_shape, mutation_rate, generations)
    c, d = ga.evolve()
    best_matrix.append(c)
    best_fitness.append(d)

9
Generation 1: Best Fitness = -6805180.720967769, Performance = 0.015147483304458071, Fidelity = 0.3194819127557398
Generation 2: Best Fitness = -6239284.377591984, Performance = 0.00791882671948814, Fidelity = 0.3760715543219749
Generation 3: Best Fitness = -5360815.879255953, Performance = 0.013438267840425927, Fidelity = 0.4639183986361368
Generation 4: Best Fitness = -5330964.049845287, Performance = 0.007133644774307862, Fidelity = 0.4669035878818265
Generation 5: Best Fitness = -7019689.028224616, Performance = 0.010810272692353826, Fidelity = 0.2980310863672657
Generation 6: Best Fitness = -6040400.7390326625, Performance = 0.019206637946558906, Fidelity = 0.39595990689009575
Generation 7: Best Fitness = -4354177.938844832, Performance = 0.008048555138659198, Fidelity = 0.5645821980669616
Generation 8: Best Fitness = -5898144.0225121435, Performance = 0.014144898860941705, Fidelity = 0.4101855836038868
Generation 9: Best Fitness = -5397277.1481845435, Performance = 0.0051742673

KeyboardInterrupt: 

## Bonus point - optimize in the hybrid encoding

In [ ]:
# Define the function. Pretty the same. the only difference is the encoding 

def evaluate_matrix_hybrid(m): # This function don't check M. make sure we input a squre matrix!
    dim = len(m) # figure out what is the dimension of the matrix
    q, r = np.linalg.qr(m) # QR decomposition. given a matrix m, we obtain q which we're sure is unitary. This serves the input of the function

    # The following part is essentially copied from the Quandela's source code
    processor = pcvl.Processor("SLOS", dim)
    processor.add(4, pcvl.BS.H()) # This is because mode 4 & 5 are the target qubit for the CCZ gate
    processor.add(0, Unitary(q))
    processor.add(4, pcvl.BS.H())

    # It seems that there is no simple way to change the encoding to hybrid mode. However, That simply means a new desired truth table. We can use the same default encoding and only change the truth table there
    processor.add_port(0, Port(Encoding.DUAL_RAIL, 'ctrl0'))
    processor.add_port(2, Port(Encoding.DUAL_RAIL, 'ctrl1'))
    processor.add_port(4, Port(Encoding.DUAL_RAIL, 'data'))

    for i in range(6, dim):
        processor.add_herald(i, 0)
    
    # Define the states and true tables for a CCNOT gate (after the Hadamard gates added on the data bit, CCZ gate has become a CCNOT gate and that's what we are going to evaluate)
    states = {
    pcvl.BasicState([1, 0, 1, 0, 1, 0]): "000",
    pcvl.BasicState([1, 0, 0, 1, 1, 0]): "010",
    pcvl.BasicState([0, 1, 1, 0, 1, 0]): "100",
    pcvl.BasicState([0, 1, 0, 1, 1, 0]): "110",
    pcvl.BasicState([1, 0, 1, 0, 0, 1]): "001",
    pcvl.BasicState([1, 0, 0, 1, 0, 1]): "011",
    pcvl.BasicState([0, 1, 1, 0, 0, 1]): "101",
    pcvl.BasicState([0, 1, 0, 1, 0, 1]): "111"
    }

    ca = pcvl.algorithm.Analyzer(processor, states)

    # What is the truth table for the CZ gate in the hybrid mode?
    truth_table = {"000": "001", "010": "010", "100": "100", "110": "111", 
                   "001": "000", "011": "011", "101": "101", "111": "110"}
    ca.compute(expected=truth_table)
    # pcvl.pdisplay(ca)
    return ca.performance, ca.fidelity.real

I used the following definition for the bonus challenge, as described in the challenge notebook.
$$
\left|s_1, s_2, s_3\right\rangle \rightarrow(-1)^{\left(s_1+s_2\right) s_3}\left|s_1, s_2, s_3\right\rangle
$$

In [6]:
# Use the same algorithm to optimized the CCZ gate in the hybrid encoding
# This is my first time ever to implement an advanced optimization algorithm to solve a real-world problem. I'm so excited! But please see if there is anything we an improve!
# I chose genetic algorithm. Not sure if this is the best choice

# Genetic Algorithm Components
class GeneticAlgorithm:
    def __init__(self, population_size, matrix_shape, mutation_rate, generations):
        self.population_size = population_size
        self.matrix_shape = matrix_shape
        self.mutation_rate = mutation_rate
        self.generations = generations
        self.population = [np.random.rand(*matrix_shape) for _ in range(population_size)]
    
    def fitness(self, matrix):
        a, b = evaluate_matrix_hybrid(Matrix(matrix))
        return 10 * a - 1e7 * (1 - b)  # Maximizing a, giving a large peanalty for b < 1
    
    def select(self):
        # Simple tournament selection
        selected = []
        for _ in range(self.population_size):
            i, j = np.random.randint(0, self.population_size, 2)
            selected.append(self.population[i] if self.fitness(self.population[i]) > self.fitness(self.population[j]) else self.population[j])
        return selected
    
    def crossover(self, parent1, parent2):
        # Single point crossover
        if np.random.rand() < 0.5:  # Swap parents to ensure diversity
            parent1, parent2 = parent2, parent1
        split = np.random.randint(1, np.prod(self.matrix_shape))
        child1_flat = np.concatenate((parent1.flatten()[:split], parent2.flatten()[split:]))
        child2_flat = np.concatenate((parent2.flatten()[:split], parent1.flatten()[split:]))
        return child1_flat.reshape(self.matrix_shape), child2_flat.reshape(self.matrix_shape)
    
    def mutate(self, matrix):
        # Random mutation
        for i in range(matrix.shape[0]):
            for j in range(matrix.shape[1]):
                if np.random.rand() < self.mutation_rate:
                    matrix[i, j] += np.random.normal(0, 1)
        return matrix
    
    def evolve(self):
        for generation in range(self.generations):
            new_population = []
            selected = self.select()
            for i in range(0, self.population_size, 2):
                parent1, parent2 = selected[i], selected[i+1]
                child1, child2 = self.crossover(parent1, parent2)
                new_population.append(self.mutate(child1))
                new_population.append(self.mutate(child2))
            self.population = new_population
            # Optional: Print best fitness in generation
            best_fitness = max(self.fitness(matrix) for matrix in self.population)
            good_matrix = max(self.population, key=self.fitness)
            p, f = evaluate_matrix(Matrix(good_matrix))
            print(f"Generation {generation+1}: Best Fitness = {best_fitness}, Performance = {p}, Fidelity = {f}")
        return self.best_solution()
    
    def best_solution(self):
        best_matrix = max(self.population, key=self.fitness)
        return best_matrix, self.fitness(best_matrix)